# Fragment Maker
A python notebook for generating new fragmentations of systems.

## Setup

In [1]:
from BigDFT import Fragments
reload(Fragments)

<module 'BigDFT.Fragments' from '/bigdft-suite-sources/IntegralYAML/Build/install/lib/python2.7/site-packages/BigDFT/Fragments.pyc'>

In [2]:
import yaml

with open("start.yaml") as ifile:
    start = yaml.load(ifile)
frag_dict = Fragments.CreateFragDict(start)

In [4]:
frag_list = Fragments.CreateFragList(frag_dict)

## Generate New List

In [10]:
merge_list = []
merge_list.append([("GLY",10), ("GLY",11)])
merge_list.append([("CLA", 568), ("WAT", 406)])
frag_list = Fragments.CreateFragList(frag_dict, merge_list)

In [11]:
sys = Fragments.System(posinp_dict=start)
for f in sys.fragments:
#     if f.id == "CLA568":
    if f.id == Fragments.SetFragId("GLY",10):
        our_frag = f
        break

In [12]:
def GetNearest(fragtuple, threshold):
    import numpy as np
    sys = Fragments.System(posinp_dict=start)
    for f in sys.fragments:
        if f.id == Fragments.SetFragId(*fragtuple):
            our_frag = f
            break
    distance_array =[Fragments.distance(our_frag, f, cell=start["cell"]) for f in sys.fragments]
    ipiv = np.argsort(distance_array)
    shell = np.where(np.array(distance_array) < threshold)[0]
    shellid = [sys.fragments[s].id for s in shell]
    return shellid

In [13]:
def CreateMergeList(neighborid):
    return [Fragments.GetFragTuple(f) for f in neighborid]

In [14]:
# merge_list.append(CreateMergeList(GetNearest(("GLY",10),3.0)))
frag_list = []
for distance in [4.0, 5.0]:
    merge_list = [CreateMergeList(GetNearest(("GLY",10), distance))]
    new_dict, temp_list = Fragments.MergeFragmentsTogether(frag_dict, merge_list)
    frag_list += temp_list
with open("merged.yaml", "w") as ofile:
    ofile.write(yaml.dump([frag[1] for frag in frag_list]))